# v3-Create-Custom-Agent-Enhanced

基于v2增强版，新增CV简历向量搜索：
- 集成Qdrant向量数据库
- 使用BGE本地嵌入模型
- 智能简历内容搜索
- 个性化面试问题

对应原版：`v3-Create-Custom-Agent.ipynb`

## 适配环境
- Python 3.11
- Qdrant向量数据库 (您自己启动的)
- BGE-small-zh本地嵌入模型
- 灵活依赖版本

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())



# 检查配置
if os.getenv('DEEPSEEK_API_KEY'):
    print("✅ DeepSeek API Key 已配置")
else:
    print("❌ 请在.env文件中配置DEEPSEEK_API_KEY")
    raise ValueError("DeepSeek API Key未配置")

if os.getenv('QDRANT_HOST'):
    print(f"🗄️ Qdrant数据库: {os.getenv('QDRANT_HOST')}:{os.getenv('QDRANT_PORT')}")
else:
    print("⚠️ Qdrant配置未找到，将使用默认localhost:6333")

✅ DeepSeek API Key 已配置
🗄️ Qdrant数据库: localhost:6333


## Load the LLM 加载LLM

In [2]:
import os
from langchain_openai import ChatOpenAI

# 使用DeepSeek作为LLM
llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0,
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
    openai_api_base="https://api.deepseek.com"
)

print("🧠 使用DeepSeek模型进行CV搜索面试")
print("🇨🇳 支持中文简历理解和分析")

🧠 使用DeepSeek模型进行CV搜索面试
🇨🇳 支持中文简历理解和分析


## 初始化Qdrant向量数据库

In [3]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Qdrant
from langchain_huggingface import HuggingFaceEmbeddings

# 从环境变量获取Qdrant配置
qdrant_host = os.getenv('QDRANT_HOST', 'localhost')
qdrant_port = int(os.getenv('QDRANT_PORT', 6333))

# 连接到Qdrant
qdrant_client = QdrantClient(host=qdrant_host, port=qdrant_port)
print(f"🗄️ 连接Qdrant: {qdrant_host}:{qdrant_port}")

# 配置BGE模型路径
model_path = os.getenv('BGE_MODEL_PATH', "BAAI/bge-small-zh-v1.5")
print(f"📁 BGE模型路径: {model_path}")

# 使用BGE嵌入模型
embeddings = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs={'device': 'cpu'},  # 如果有GPU可以改为'cuda'
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ 向量数据库和嵌入模型初始化完成")

🗄️ 连接Qdrant: localhost:6333
📁 BGE模型路径: C:\Users\guohaoyu\.cache\huggingface\hub\models--BAAI--bge-small-zh-v1.5\snapshots\7999e1d3359715c523056ef9478215996d62a620
✅ 向量数据库和嵌入模型初始化完成


## Define Enhanced Tools 定义增强工具

In [4]:
from langchain.agents import tool
import time
import os
import shutil

@tool
def generate_unique_timestamp():
    """
    生成唯一的时间戳。输入始终为空字符串。

    Returns:
        int: 唯一的时间戳，以毫秒为单位。
    """
    timestamp = int(time.time() * 1000)
    return timestamp

@tool
def create_folder(folder_name):
    """
    根据给定的文件夹名创建文件夹。

    Args:
        folder_name (str): 要创建的文件夹的名称。

    Returns:
        str: 创建的文件夹的路径。
    """
    try:
        folder_path = os.path.join("chat_history", folder_name)
        os.makedirs(folder_path, exist_ok=True)
        return os.path.abspath(folder_path)
    except OSError as e:
        print(f"创建文件夹失败：{e}")
        return None

@tool
def delete_temp_folder():
    """
    删除 chat_history 文件夹下的 temp 文件夹。输入始终为空字符串。

    Returns:
        bool: 如果成功删除则返回 True，否则返回 False。
    """
    temp_folder = "chat_history/temp"
    try:
        if os.path.exists(temp_folder):
            shutil.rmtree(temp_folder)
            print("成功删除 temp 文件夹。")
        return True
    except Exception as e:
        print(f"删除 temp 文件夹失败：{e}")
        return False

In [5]:
@tool
def copy_chat_history(interview_id: str) -> str:
    """
    将 chat_history/temp 文件夹中的 chat_history.txt 文件复制到指定面试ID文件夹中。
    """
    temp_folder = os.path.join("chat_history", "temp")
    interview_folder = os.path.join("chat_history", interview_id)

    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法完成复制操作。"

    source_file = os.path.join(temp_folder, 'chat_history.txt')
    destination_file = os.path.join(interview_folder, 'chat_history.txt')

    if os.path.exists(source_file):
        shutil.copyfile(source_file, destination_file)
        return f"已将 chat_history.txt 复制到面试ID为 {interview_id} 的文件夹中。"
    else:
        return "临时聊天记录文件不存在。"

@tool
def read_chat_history(interview_id: str) -> str:
    """
    读取指定面试ID文件夹下的聊天记录内容。
    """
    interview_folder = os.path.join("chat_history", interview_id)

    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法读取聊天记录。"

    chat_history_file = os.path.join(interview_folder, 'chat_history.txt')
    if os.path.exists(chat_history_file):
        with open(chat_history_file, 'r', encoding='utf-8') as file:
            return file.read()
    else:
        return "聊天记录文件不存在。"

@tool
def generate_markdown_file(interview_id: str, interview_feedback: str) -> str:
    """
    生成面试报告的Markdown文件。
    """
    interview_folder = os.path.join("chat_history", interview_id)

    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法生成 Markdown 文件。"

    markdown_file_path = os.path.join(interview_folder, "面试报告.md")

    try:
        with open(markdown_file_path, 'w', encoding='utf-8') as file:
            file.write("# 面试报告\n\n")
            file.write("## 面试反馈：\n\n")
            file.write(interview_feedback)
            file.write("\n\n")

            # 添加面试记录
            chat_history_file_path = os.path.join(interview_folder, "chat_history.txt")
            if os.path.exists(chat_history_file_path):
                file.write("## 面试记录：\n\n")
                with open(chat_history_file_path, 'r', encoding='utf-8') as chat_file:
                    for line in chat_file:
                        file.write(line.rstrip('\n') + '\n\n')

        return f"已生成 Markdown 文件: {markdown_file_path}"
    except Exception as e:
        return f"生成 Markdown 文件时出错: {str(e)}"

## 增强工具：Qdrant向量搜索

In [6]:
@tool
def find_most_relevant_block_from_cv_qdrant(query: str) -> str:
    """
    使用Qdrant向量数据库搜索简历中与查询最相关的内容块。
    当你需要根据职位描述（JD）中的技能关键词去简历文本中找到相关内容时，就可以调用这个函数。
    
    Args:
        query (str): 搜索查询，通常是技能关键词或技术点
    
    Returns:
        str: 简历中最相关的内容块
    """
    try:
        # 创建Qdrant向量存储
        vector_store = Qdrant(
            client=qdrant_client,
            collection_name="cv_collection",
            embeddings=embeddings,
        )
        
        # 搜索相关内容
        results = vector_store.similarity_search(query, k=2)
        
        if results:
            # 返回最相关的内容
            return results[0].page_content
        else:
            return "未找到相关的简历内容。"
            
    except Exception as e:
        print(f"find_most_relevant_block_from_cv_qdrant()发生错误：{e}")
        return "搜索简历内容时发生错误。"

In [7]:
# 定义所有工具
tools = [
    generate_unique_timestamp, 
    create_folder, 
    delete_temp_folder,
    copy_chat_history, 
    read_chat_history, 
    generate_markdown_file, 
    find_most_relevant_block_from_cv_qdrant
]

## 数据预处理：JD和CV解析

In [8]:
# 解析JD文件（如果需要重新解析）
from utils_enhanced import parse_jd_to_json, read_json

# 可选：重新解析JD
# jd_file_path = "data/jd.txt"
# jd_json_file_path = parse_jd_to_json(llm, jd_file_path)

# 读取已解析的JD
jd_json_file_path = "data/jd.json"
if os.path.exists(jd_json_file_path):
    jd_dict = read_json(jd_json_file_path)
    job_title = jd_dict.get('基本信息', {}).get('职位', 'Python工程师')
    job_key_skills = jd_dict.get('专业技能/知识/能力', [])
    print(f"职位：{job_title}")
    print(f"专业技能/知识/能力：{job_key_skills}")
else:
    job_title = "Python工程师 (AI应用方向)"
    job_key_skills = ["Python", "AI", "机器学习"]

职位：AI工程师
专业技能/知识/能力：['Python开发', '机器学习框架(TensorFlow, PyTorch)', 'LangChain', '向量数据库', 'Agent开发', 'DeepSeek等大语言模型', 'AI应用系统开发', '智能对话机器人设计与实现', '模型性能优化', '用户体验优化', 'AI技术落地']


In [9]:
# 解析CV文件并存储到Qdrant
from utils_enhanced import parse_cv_to_md
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_core.documents import Document

# 解析CV为Markdown（如果需要重新解析）
# cv_file_path = "data/cv.txt"
# result = parse_cv_to_md(llm, cv_file_path)

# 读取CV Markdown文件
cv_md_path = "data/cv.md"
if os.path.exists(cv_md_path):
    with open(cv_md_path, 'r', encoding='utf-8') as file:
        markdown_text = file.read()

    # 分割文档
    docs = []
    headers_to_split_on = [
        ("#", "Title 1"),
        ("##", "Title 2"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    split_docs = markdown_splitter.split_text(markdown_text)
    
    for split_doc in split_docs:
        metadata = split_doc.metadata
        title_str = f"# {metadata.get('Title 1', 'None')}\n## {metadata.get('Title 2', 'None')}\n"
        page_content = title_str + split_doc.page_content.strip()
        doc = Document(
            page_content=page_content,
            metadata=metadata
        )
        docs.append(doc)

    print(f"解析了 {len(docs)} 个简历文档块")
else:
    print("CV Markdown文件不存在，请先解析CV文件")
    docs = []

解析了 11 个简历文档块


In [19]:

# 存储文档到Qdrant
if docs:
    try:
        # 先尝试创建集合（如果不存在）
        from qdrant_client.models import Distance, VectorParams
        
        collection_name = "cv_collection"
        
        # 检查集合是否存在
        try:
            collection_info = qdrant_client.get_collection(collection_name)
            print(f"✅ 集合 '{collection_name}' 已存在")
        except Exception:
            # 集合不存在，创建新集合
            print(f"📝 创建新集合: {collection_name}")
            qdrant_client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(
                    size=512,  # BGE-small-zh-v1.5 的向量维度
                    distance=Distance.COSINE
                )
            )
            print(f"✅ 成功创建集合: {collection_name}")
        
        # 现在创建 vector_store 并添加文档
        vector_store = Qdrant(
            client=qdrant_client,
            collection_name=collection_name,
            embeddings=embeddings,
        )
        
        # 检查集合是否已有文档
        try:
            test_results = vector_store.similarity_search("Python", k=1)
            if test_results:
                print("✅ 集合已存在文档，无需重新添加")
            else:
                # 集合为空，添加文档
                vector_store.add_documents(docs)
                print(f"✅ 成功将 {len(docs)} 个文档存储到Qdrant")
        except Exception as e:
            # 如果搜索失败，直接添加文档
            vector_store.add_documents(docs)
            print(f"✅ 成功将 {len(docs)} 个文档存储到Qdrant")
            
    except Exception as e:
        print(f"存储到Qdrant时出错: {e}")
        print("请确保Docker中的Qdrant服务正在运行")
        print("可以尝试重启Qdrant服务：docker restart qdrant")

✅ 集合 'cv_collection' 已存在
✅ 集合已存在文档，无需重新添加


## Create Enhanced Prompt 创建增强提示

In [11]:
system_prompt = f"""
## Role and Goals
- 你是所招岗位"{job_title}"的技术专家，同时也作为技术面试官向求职者提出技术问题，专注于考察应聘者的专业技能、知识和能力。
- 这里是当前岗位所需的专业技能、知识和能力："{job_key_skills}"，你应该重点围绕这些关键项提出你的问题。
- 你拥有高级的简历内容搜索能力，能够根据技术点智能匹配求职者的相关经验。
- 你严格遵守面试流程进行面试。

## Interview Workflow
1. 当应聘者说开始面试后，
    1.1 你要依据当前时间生成一个新的时间戳作为面试ID（只会在面试开始的时候生成面试ID，其他任何时间都不会）
    1.2 以该面试ID为文件夹名创建本地文件夹（只会在面试开始的时候创建以面试ID为名的文件夹，其他任何时间都不会）
    1.3 删除存储聊天记录的临时文件夹
    1.4 输出该面试ID给应聘者，并结合当前技术点、与技术点相关的简历内容，提出你的第一个基础技术问题。
2. 接收应聘者的回答后，
    2.1 检查应聘者的回答是否有效
        2.1.1 如果是对面试官问题的正常回答（无论回答的好不好，还是回答不会，都算正常回答），就跳转到2.2处理
        2.1.2 如果是与面试官问题无关的回答（胡言乱语、辱骂等），请警告求职者需要严肃对待面试，跳过2.2，再次向求职者提出上次的问题。
    2.2 如果应聘者对上一个问题回答的很好，就基于当前知识点提出一个更深入一点的问题；
        如果应聘者对上一个问题回答的一般，就基于当前知识点提出另一个角度的问题；
        如果应聘者对上一个问题回答的不好，就基于当前知识点提出一个更简单一点的问题；
        如果应聘者对上一个问题表示不会、不懂、一点也回答不了，就换一个与当前知识点不同的知识点进行技术提问。
3. 当应聘者想结束面试或当应聘者想要面试报告，
    3.1 从临时文件夹里复制一份聊天记录文件到当前面试ID文件夹下。
    3.2 读取当前面试ID文件夹下的聊天记录，基于聊天记录、从多个角度评估应聘者的表现、生成一个详细的面试报告。
    3.3 调用工具生成一个面试报告的markdown文件到当前面试ID文件夹下
    3.4 告知应聘者面试已结束，以及面试报告的位置。
    
## Enhanced Features
- 你每次提出的技术问题，都需要结合从JD里提取的技术点和与技术点相关的简历内容，当你需要获取`与技术点相关的简历内容`时，请调用智能搜索工具。
- 你的问题应该既考虑岗位需求，又结合求职者的实际经验背景，做到个性化和针对性。
- 使用向量搜索技术，你能够精准找到简历中与当前讨论技术点最相关的项目经验或技能描述。

## Output Constraints
- 你一次只会问一个面试问题。
- 你发送给应聘者的信息中，一定不要解答你提出的面试问题，只需要有简短的反馈和提出的新问题。
- 充分利用你的智能搜索能力，让面试更加个性化和专业。
"""

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

print("✅ 增强版面试提示创建完成")

✅ 增强版面试提示创建完成


## Create the Enhanced Agent 创建增强代理

In [13]:
# 绑定工具到LLM
llm_with_tools = llm.bind_tools(tools)

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ 增强版Agent执行器创建完成")

✅ 增强版Agent执行器创建完成


## Run the Enhanced Agent 运行增强代理

In [14]:
from utils_enhanced import save_chat_history
from langchain_core.messages import AIMessage, HumanMessage
import os

chat_history = []

user_input = "开始面试"
print(f"应聘者: {user_input}")
print("=" * 50)

while True:
    try:
        result = agent_executor.invoke({"input": user_input, "chat_history": chat_history})
        print(f"面试官: {result['output']}")
        print("=" * 50)
        
        chat_history.extend([
            HumanMessage(content=user_input),
            AIMessage(content=result["output"]),
        ])
        
        # 存储聊天记录到临时文件夹
        temp_folder = "chat_history/temp"
        os.makedirs(temp_folder, exist_ok=True)
        save_chat_history(chat_history, temp_folder)

        # 获取用户下一条输入
        user_input = input("\n应聘者: ").strip()
        
        # 检查退出命令（更严格的匹配）
        exit_commands = ["exit", "退出", "结束面试", "quit", "bye", "再见"]
        if user_input.lower() in exit_commands or user_input == "":
            print("\n🎯 面试结束！")
            print("感谢您的参与，祝您求职顺利！")
            
            # 生成最终面试报告
            try:
                from datetime import datetime
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                final_folder = f"chat_history/{timestamp}"
                os.makedirs(final_folder, exist_ok=True)
                save_chat_history(chat_history, final_folder)
                print(f"📄 面试记录已保存到: {final_folder}")
            except Exception as e:
                print(f"保存面试记录时出错: {e}")
            
            break
            
    except KeyboardInterrupt:
        print("\n\n🛑 面试被中断")
        break
    except Exception as e:
        print(f"发生错误: {e}")
        print("请检查Qdrant服务是否正常运行，以及模型路径是否正确。")
        
        # 询问是否继续
        continue_choice = input("是否继续面试？(y/n): ").strip().lower()
        if continue_choice in ['n', 'no', '否', '不']:
            break
        else:
            user_input = input("请重新输入: ").strip()

print("面试系统已退出。")

应聘者: 开始面试


> Entering new AgentExecutor chain...

Invoking: `generate_unique_timestamp` with `{}`


1753926172875
Invoking: `create_folder` with `{'folder_name': '1712345678901'}`


C:\learn\7-22\llm-developing-mock-interview\chat_history\1712345678901
Invoking: `delete_temp_folder` with `{}`


成功删除 temp 文件夹。
True面试ID：1712345678901

请介绍一下你在Python开发方面的经验，特别是在机器学习或AI相关项目中的应用。

> Finished chain.
面试官: 面试ID：1712345678901

请介绍一下你在Python开发方面的经验，特别是在机器学习或AI相关项目中的应用。



应聘者:  没有




> Entering new AgentExecutor chain...
好的，那我们换一个话题。你在简历中提到过使用机器学习框架（如TensorFlow或PyTorch）的经验吗？如果有，可以分享一下你在这些框架中的具体项目或任务吗？

> Finished chain.
面试官: 好的，那我们换一个话题。你在简历中提到过使用机器学习框架（如TensorFlow或PyTorch）的经验吗？如果有，可以分享一下你在这些框架中的具体项目或任务吗？



应聘者:  结束面试



🎯 面试结束！
感谢您的参与，祝您求职顺利！
📄 面试记录已保存到: chat_history/20250731_094402
面试系统已退出。


## 测试向量搜索功能

In [16]:
# 测试向量搜索功能
test_queries = ["Python开发经验", "机器学习项目", "LangChain", "数据分析"]

for query in test_queries:
    print(f"\n搜索: {query}")
    result = find_most_relevant_block_from_cv_qdrant.invoke(query)
    print(f"结果: {result[:200]}...")  # 只显示前200个字符


搜索: Python开发经验
结果: # 自我评价
## None
- 自我反思：具有深厚的计算机科学理论基础和丰富的NLP实践经验，擅长通过技术创新解决复杂问题。
- 专业技能：精通Python编程、PyTorch、LangChain、Agent架构；熟悉NLP相关算法和预训练模型，如Transformer、BERT、GPT等。熟练使用Pandas、Numpy、Sklearn等数据科学和机器学习库。
- 个人优势：具有在大型互联网公...

搜索: 机器学习项目
结果: # 兴趣爱好
## None
- 机器学习和人工智能领域的技术博客撰写
- 参与开源NLP项目和技术社区活动
- 长跑和羽毛球...

搜索: LangChain
结果: # 项目经历
## 基于LangChain的智能问答系统（在线教育平台）
- 任务：为一家在线教育平台设计并实现一个基于LangChain的智能问答系统，旨在提供即时、准确的学术支持和课程咨询服务。
- 难点：整合广泛的教育资源和文献，提高问答系统对学术术语的理解能力和回答质量。
- 解决方案：与教育专家合作开发定制化的知识图谱，利用LangChain进行高效的信息检索和生成，确保回答的专业性和准...

搜索: 数据分析
结果: # 项目经历
## 舆情分析系统（金融领域）
- 任务：为一家金融公司开发一个舆情分析系统，旨在从大量的新闻报道和社交媒体帖子中识别并分析与公司相关的情绪倾向。
- 难点：处理和分析海量的非结构化文本数据，准确识别出与公司相关的命名实体，如股票代码、产品名称等，并对文本进行情感倾向分析。
- 解决方案：采用自然语言处理技术进行命名实体识别，利用机器学习模型进行情感分析。通过Pandas进行数据清洗...
